# Preprocessing

In [3]:
import pandas as pd
import numpy as np  
from sklearn import model_selection
from sklearn import ensemble
from sklearn import svm
from sklearn import neighbors
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.ensemble import VotingClassifier

df = pd.read_csv('ENB2012_data.csv', sep=";")

df.head()
df.info()

for col in df.columns:
    if df[col].dtype == object:
        try:
            df[col] = df[col].astype(np.float64)
        except ValueError:
            pass



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   X1      768 non-null    float64
 1   X2      768 non-null    float64
 2   X3      768 non-null    float64
 3   X4      768 non-null    float64
 4   X5      768 non-null    float64
 5   X6      768 non-null    int64  
 6   X7      768 non-null    float64
 7   X8      768 non-null    int64  
 8   Y1      768 non-null    float64
 9   Y2      768 non-null    float64
dtypes: float64(8), int64(2)
memory usage: 60.1 KB


In [4]:
# Analyse corrélation entre les variable de df (corrélation de Pearson)
pearson = df.corr()
pearson

# Variable explicative les plus corrèlées aux variables cibles
corrTarget = pearson[['Y1', 'Y2']].abs().sort_values(by=['Y1', 'Y2'], ascending=False)
corrTarget

print('On constate que les variables Y1, Y2 et X5 sont importante pour prédire nos deux cibles')

On constate que les variables Y1, Y2 et X5 sont importante pour prédire nos deux cibles


In [5]:
# Nv col total charges chauffage et climatisation
df['charges_sum'] = df['Y1']+ df['Y2']

# Recherche quartiles
quartile = df['charges_sum'].quantile([0.25, 0.5, 0.75])


# Découpage
charges_classes = pd.cut(df['charges_sum'], bins = [-float('inf'), quartile[0.25], quartile[0.5], quartile[0.75], float('inf')],
                         labels=[0, 1, 2, 3])



In [6]:
# Stockage données explicative
df_X = df.drop(columns =['charges_sum'])

# Séparation ensemble test
X_train, X_test, y_train, y_test = train_test_split(df_X, charges_classes, test_size=0.2)

# Centralisation et réduction des varaibles explicative
scaler = preprocessing.StandardScaler().fit(X_train)

X_train_scaled = scaler.transform(X_train)

X_test_scaled = scaler.transform(X_test)

# Sélection du modèle

# K plus proches voisins

In [27]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

# Hyperparamètre
param = {'n_neighbors': range(2, 41)}

# Classifieur KNN
knn = KNeighborsClassifier()

# Meilleirs hyperparamètre recherche
grid_search = GridSearchCV(knn, param)
grid_search.fit(X_train_scaled, y_train)

BParams = grid_search.best_params_

BParams

print('Meilleur param = 3')

Meilleur param = 3


In [29]:
# Application modèle

knnB = KNeighborsClassifier(**BParams)
knnB.fit(X_train_scaled, y_train)

y_pred = knnB.predict(X_test_scaled)

confusion = pd.crosstab(y_test, y_pred, rownames=['Classes réelles'], colnames=['Classes prédites'])
confusion

Classes prédites,0,1,2,3
Classes réelles,,,,
0,38,2,0,0
1,1,42,2,0
2,0,2,28,2
3,0,0,2,35


In [31]:
score = knnB.score(X_test_scaled, y_test)
score

0.9285714285714286

# SVM

In [8]:
from sklearn.svm import SVC 
from sklearn.model_selection import GridSearchCV
# Hyperparamètre
param = {'kernel': ['linear', 'sigmoid', 'rbf'], 'C': [0.1, 1, 10, 30]}

# Classifieur SVM
svm = SVC()

#Meilleurs hyperpamètre recherche
grid_search = GridSearchCV(svm, param)
grid_search.fit(X_train_scaled, y_train)

BParams = grid_search.best_params_

BParams

print("Meilleur param 'C': 10, 'kernel': 'linear' ")

Meilleur param 'C': 10, 'kernel': 'linear' 


In [35]:
#Application du modèle
svmB = SVC(**BParams)
svmB.fit(X_train_scaled, y_train)
y_pred = svmB.predict(X_test_scaled)

confusion = pd.crosstab(y_test, y_pred, rownames=['Classe réalle'], colnames=['Classe prédite'])
confusion

Classe prédite,0,1,2,3
Classe réalle,,,,
0,38,2,0,0
1,0,45,0,0
2,0,1,31,0
3,0,0,0,37


In [37]:
score = svmB.score(X_test_scaled, y_test)
score

0.9805194805194806

# Random Forest


In [12]:
from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection import GridSearchCV 

# Hyperparamètre
param = {'max_features': ["sqrt", "log2", None], 'min_samples_split': range(2, 31, 2)}

# Classifieur rf
rf = RandomForestClassifier()

#Meilleurs hyperpamètre recherche
grid_search = GridSearchCV(rf, param)
grid_search.fit(X_train_scaled, y_train)

BParams = grid_search.best_params_

BParams

print("Meilleur param : {'max_features': 'sqrt', 'min_samples_split': 10}") 

Meilleur param : {'max_features': 'sqrt', 'min_samples_split': 10}


In [14]:
#Application du modèle
rfB = RandomForestClassifier(**BParams)
rfB.fit(X_train_scaled, y_train)
y_pred = rfB.predict(X_test_scaled)

confusion = pd.crosstab(y_test, y_pred, rownames=['Classe réalle'], colnames=['Classe prédite'])
confusion

Classe prédite,0,1,2,3
Classe réalle,,,,
0,36,0,0,0
1,0,40,0,0
2,0,0,42,0
3,0,0,0,36


In [15]:
score = rfB.score(X_test_scaled, y_test)
score

1.0

- Random Forest : 1.0
- SVM : 0.9805194805194806
- KNN : 0.9285714285714286


Le modèle Random Forest a obtenu un score de précision de 1.0, ce qui signifie qu'il a correctement prédit toutes les observations dans l'ensemble de test. Par conséquent, dans ce cas, le modèle Random Forest offre la meilleure précision.

On aurez pu entraîner un algorithme de régression à la place si l'objectif était de prédire une valeur continue plutôt qu'une classification des charges énergétiques.

Cela dépend de la nature du problème et de l'objectif de l'analyse.